In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Burundi_Code_2017_penal.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
# data

In [9]:
len(data)

128

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)
# print(docs[200])
# print("================================================")
print("Total number of documents: ",len(docs))

Total number of documents:  436
